In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session
import numpy as np
import pandas as pd
from pandas import Series, DataFrame
import matplotlib.pyplot as plt
import re as re

In [ ]:
train_df = pd.read_csv('/kaggle/input/GiveMeSomeCredit/cs-training.csv')
test_df = pd.read_csv('/kaggle/input/GiveMeSomeCredit/cs-test.csv')

# 1.查看训练集和测试集相关信息

In [ ]:
print(train_df.info())
train_df.head(5)

In [ ]:
print(test_df.info())
test_df.head(5)

In [ ]:
train_df.describe()

根据上述信息进行数据清洗

# 2.数据处理

根据上述describe观察到seriousDlqin2yrs的分布不均匀，绘制柱状图验证

In [ ]:
import seaborn as sns
plt.figure()
sns.countplot('SeriousDlqin2yrs',data=train_df)

In [ ]:
train_df.loc[train_df['age'] == 0, 'age'] = train_df['age'].median()
test_df.loc[test_df['age'] == 0, 'age'] = test_df['age'].median()

分别采用均值对'NumberOfDependents'和'MonthlyIncome'进行缺失值处理


In [ ]:
train_df['NumberOfDependents'] = train_df['NumberOfDependents'].replace(np.nan,train_df['NumberOfDependents'].median())
test_df['NumberOfDependents'] = test_df['NumberOfDependents'].replace(np.nan,test_df['NumberOfDependents'].median())

In [ ]:
train_df['MonthlyIncome'] = train_df['MonthlyIncome'].replace(np.nan,train_df['MonthlyIncome'].mean())
test_df['MonthlyIncome'] = test_df['MonthlyIncome'].replace(np.nan,test_df['MonthlyIncome'].mean())

观察变量之间的相关性

In [ ]:
corr = train_df.corr()
plt.figure(figsize=(19, 15))
sns.heatmap(corr, annot=True,cmap='Blues')

可见对于'NumberOfTime30-59DaysPastDueNotWorse', 
          'NumberOfTime60-89DaysPastDueNotWorse',
          'NumberOfTimes90DaysLate'，相关性过高，需要处理

In [ ]:
plt.figure(figsize=(19, 12)) 
train_df[['NumberOfTime30-59DaysPastDueNotWorse', 
          'NumberOfTime60-89DaysPastDueNotWorse',
          'NumberOfTimes90DaysLate']].boxplot()
plt.show()

In [ ]:
def replacehigh(column):
    new = []
    newval = column.median()
    for i in column:
        if (i>80):
            new.append(newval)
        else:
            new.append(i)
    return new

train_df['NumberOfTime30-59DaysPastDueNotWorse'] = replacehigh(train_df['NumberOfTime30-59DaysPastDueNotWorse'])
train_df['NumberOfTimes90DaysLate'] = replacehigh(train_df['NumberOfTimes90DaysLate'])
train_df['NumberOfTime60-89DaysPastDueNotWorse'] = replacehigh(train_df['NumberOfTime60-89DaysPastDueNotWorse'])

test_df['NumberOfTime30-59DaysPastDueNotWorse'] = replacehigh(test_df['NumberOfTime30-59DaysPastDueNotWorse'])
test_df['NumberOfTimes90DaysLate'] = replacehigh(test_df['NumberOfTimes90DaysLate'])
test_df['NumberOfTime60-89DaysPastDueNotWorse'] = replacehigh(test_df['NumberOfTime60-89DaysPastDueNotWorse'])

In [ ]:
corr = train_df.corr()
plt.figure(figsize=(19, 15))
sns.heatmap(corr, annot=True,cmap='Blues')

可见在处理之后相关性减弱

# 3.建立模型

In [ ]:
X = train_df.drop(['SeriousDlqin2yrs', 'Unnamed: 0'],axis=1)
y = train_df['SeriousDlqin2yrs']
XT = test_df.drop(['SeriousDlqin2yrs', 'Unnamed: 0'],axis=1)
YT = test_df['SeriousDlqin2yrs']

划分测试集和训练集

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split, cross_val_predict
from sklearn.metrics import roc_curve, roc_auc_score
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier

X_train, X_test, y_train, y_test = train_test_split(X,y,random_state=111)


利用梯度提升法进行预测结果

In [ ]:
def plot_roc_curve(fpr, tpr, label=None):
    plt.figure(figsize=(12,10))
    plt.plot(fpr, tpr, linewidth=2, label=label)
    plt.plot([0,1],[0,1], "k--") 
    plt.axis([0,1,0,1])
    plt.xlabel("False Positive Rate")
    plt.ylabel("True Positive rate")
gbc_clf = GradientBoostingClassifier(n_estimators=300, learning_rate=0.05, max_depth=8, random_state=112)
gbc_clf.fit(X_train, y_train)
gbc_clf_proba = gbc_clf.predict_proba(X_train)
gbc_clf_scores = gbc_clf_proba[:, 1]
fpr_gbc, tpr_gbc, thresh_gbc = roc_curve(y_train, gbc_clf_scores)
plot_roc_curve(fpr_gbc, tpr_gbc)
print('AUC Score:', roc_auc_score(y_train, gbc_clf_scores))

In [ ]:
gbc_val_proba = gbc_clf.predict_proba(X_test)
gbc_val_scores = gbc_val_proba[:, 1]
print('AUC score:', roc_auc_score(y_test, gbc_val_scores))

接下来进行调参

In [ ]:
esmi=np.linspace(100,300,21)
auc_score_train=[]
auc_score_test=[]
for e in esmi:
    print(e)
    gbc_clf_submission = GradientBoostingClassifier(n_estimators=int(e), learning_rate=0.05 ,max_depth=4,  random_state=40)
    gbc_clf_submission.fit(X_train,y_train)
    gbc_clf_proba = gbc_clf_submission.predict_proba(X_train)
    gbc_clf_scores = gbc_clf_proba[:,1]
    gbc_val_proba = gbc_clf_submission.predict_proba(X_test)
    gbc_val_scores = gbc_val_proba[:,1]
    fpr_gbc, tpr_gbc, thresh_gbc = roc_curve(y_train, gbc_clf_scores)
    auc_score_train.append(roc_auc_score(y_train, gbc_clf_scores))
    auc_score_test.append(roc_auc_score(y_test, gbc_val_scores))
    
fig=plt.figure()
fig.set_size_inches(12,6)
ax=fig.add_subplot(1,2,1)
ax.plot(esmi,auc_score_train,marker='+')
ax.set_xlabel("estimation")
ax.set_ylim(0,1)
ax.set_ylabel('auc_train')

ax=fig.add_subplot(1,2,2)
ax.plot(esmi,auc_score_test,marker='+')
ax.set_xlabel("estimation")
ax.set_ylim(0,1)
ax.set_ylabel('auc_test')


In [ ]:
depth=np.linspace(2,10,9)
auc_score_train=[]
auc_score_test=[]
for d in depth:
    print(d)
    gbc_clf_submission = GradientBoostingClassifier(n_estimators=200, learning_rate=0.05 ,max_depth=int(d),  random_state=40)
    gbc_clf_submission.fit(X_train,y_train)
    gbc_clf_proba = gbc_clf_submission.predict_proba(X_train)
    gbc_clf_scores = gbc_clf_proba[:,1]
    gbc_val_proba = gbc_clf_submission.predict_proba(X_test)
    gbc_val_scores = gbc_val_proba[:,1]
    fpr_gbc, tpr_gbc, thresh_gbc = roc_curve(y_train, gbc_clf_scores)
    auc_score_train.append(roc_auc_score(y_train, gbc_clf_scores))
    auc_score_test.append(roc_auc_score(y_test, gbc_val_scores))
    
fig=plt.figure()
fig.set_size_inches(12,6)
ax=fig.add_subplot(1,2,1)
ax.plot(depth,auc_score_train,marker='+')
ax.set_xlabel("estimation")
ax.set_ylim(0,1)
ax.set_ylabel('auc_train')

ax=fig.add_subplot(1,2,2)
ax.plot(depth,auc_score_test,marker='+')
ax.set_xlabel("estimation")
ax.set_ylim(0,1)
ax.set_ylabel('auc_test')

In [ ]:
def plot_feature_importances(model):
    plt.figure(figsize=(10,8))
    n_features = X.shape[1]
    plt.barh(range(n_features), model.feature_importances_, align='center')
    plt.yticks(np.arange(n_features), X.columns)
    plt.xlabel('Feature importance')
    plt.ylabel('Feature')
    plt.ylim(-1, n_features)
plot_feature_importances(gbc_clf)

In [ ]:
submission_proba = gbc_clf_submission.predict_proba(XT)
submission_scores = submission_proba[:, 1]

In [ ]:
ids = np.arange(1, 101504)
submission = pd.DataFrame( {'Id': ids, 'Probability': submission_scores})
submission.to_csv('submission.csv', index=False)